In [ ]:
%%capture
! pip install deeplabcut
import deeplabcut

In [ ]:
! cp -r /content/drive/MyDrive/5561/baseline ./baseline

In [ ]:
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/train_cropped.zip train_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/val_cropped.zip val_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/test_cropped.zip test_cropped.zip

In [ ]:
%%capture
! unzip train_cropped.zip
! unzip val_cropped.zip
! unzip test_cropped.zip

In [ ]:
import os
import shutil

num_train = 66917
num_val = 22306

for i in range(num_train):
  filename = f"train_cropped_{str(i).rjust(7, '0')}.jpg"
  shutil.move("train_cropped/" + filename, "baseline/labeled-data/baseline/" + filename)

for i in range(num_val):
  filename = f"val_cropped_{str(i).rjust(7, '0')}.jpg"
  shutil.move("val_cropped/" + filename, "baseline/labeled-data/baseline/" + filename)

In [ ]:
path_config_file = "/content/baseline/config.yaml"

deeplabcut.create_training_dataset(path_config_file, augmenter_type="imgaug")

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([17822, 50453, 32018, ..., 78661, 20736,  3126]),
   array([11229, 63645, 22163, ..., 20730, 82348, 69873])))]

In [ ]:
#assuming you work with 
shuffle=1
trainingsetindex=0

#get path to the model configuration files
trainposeconfigfile,testposeconfigfile,snapshotfolder=deeplabcut.return_train_network_path(path_config_file, shuffle=shuffle ,trainingsetindex=trainingsetindex)

#load the training config file and change various parameters:
cfg_dlc=deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)

# cfg_dlc['scale_jitter_lo']= 0.5
# cfg_dlc['scale_jitter_up']=1.5

# cfg_dlc['augmentationprobability']=.5
# cfg_dlc['batch_size']=8 #pick that as large as your GPU can handle it
# cfg_dlc['elastic_transform']=True
# cfg_dlc['rotation']=180
# cfg_dlc['covering']=True
# cfg_dlc['motion_blur'] = True
# cfg_dlc['optimizer'] ="adam"
# cfg_dlc['dataset_type']='imgaug'
cfg_dlc['multi_step']=[[0.001, 1030000]]
cfg_dlc["init_weights"] = "/content/baseline/dlc-models/iteration-0/baselineMar19-trainset95shuffle1/train/snapshot-403000"

#save it and then train the model!
deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile,cfg_dlc)

In [ ]:
deeplabcut.train_network(path_config_file, max_snapshots_to_keep=5, displayiters=100, saveiters=1000, maxiters=1030000, allow_growth=True)

In [ ]:
deeplabcut.analyze_videos(path_config_file,
                          ['/content/test_cropped'], 
                          videotype='.jpg', 
                          # save_as_csv=True, 
                          destfolder="/content/test_predictions")